# Tulap Mechanism

In [2]:
pip install opendp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 39.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import scipy.optimize
from scipy.stats import binom
import math
from postprocessors import 
import opendp.prelude as dp
dp.enable_features("contrib")

In [8]:
# binomial data
import numpy as np


reps = 10  
n = 10      
truth = 0.5 


X = np.random.binomial(n, truth, reps)


In [10]:

ump_result = binomial_tulap.ump_test(alpha, tail)
print("UMP Test Result:", ump_result)

In [11]:
from opendp.mod import enable_features
enable_features("contrib")

In [12]:
from opendp.transformations import make_split_dataframe, make_select_column

In [ ]:
from opendp.transformations import make_split_dataframe, make_select_column
age_trans = (
    make_split_dataframe(separator=",", col_names=metadata["column_names"]) >>
    make_select_column("age", str)
)

In [ ]:
epsilon = 1
delta = 0.01
theta = 0.5  
size = len(data)
alpha = 0.05
tail = "left"


binomial_tulap = Tulap(data, epsilon, delta, theta, size)


ump_result = binomial_tulap.ump_test(alpha, tail)
print("UMP Test Result:", ump_result)

In [ ]:
# Compute a two-sided confidence interval
ci_twoside_result = binomial_tulap.CI(alpha)
print("Two-sided Confidence Interval:", ci_twoside_result)

# Compute a one-sided confidence interval (lower)
ci_oneside_lower_result = binomial_tulap.CI(alpha, "lower")
print("One-sided (Lower) Confidence Interval:", ci_oneside_lower_result)

In [ ]:
# Calculate a two-sided p-value
p_value_twoside = binomial_tulap.p_value()
print("Two-sided P-value:", p_value_twoside)

# Calculate a one-sided p-value (left tail)
p_value_oneside_left = binomial_tulap.p_value("left")
print("One-sided (Left) P-value:", p_value_oneside_left)
